<a href="https://colab.research.google.com/github/astrfo/RS_init/blob/main/RS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RS 実装

[満足化を通じた最適な自律的探索] https://www.jstage.jst.go.jp/article/pjsai/JSAI2018/0/JSAI2018_1Z304/_article/-char/ja/

[論文要約] https://colab.research.google.com/drive/199SxVBGDdkuzSzU9pjVP9BfMPzaypLfj#scrollTo=UUSaBpguVsBB




In [195]:
import sys
import numpy as np
import random
import matplotlib.pyplot as plt
%matplotlib inline

In [196]:
class Environment(object):

    def __init__(self, K):
        self._K = K
        self.prob = [np.random.rand() for _ in range(K)]   #一様分布からサンプリング
        # print(f'{self.prob}')

    def play(self, arm):
        if self.prob[arm] > random.random():
            return 1
        else:
            return 0

In [197]:
class Agent(object):

    def __init__(self, K):
        self._K = K
        self._alpha = None
        self._V = None
        self._n = None
        self._N = None
        self.RS = None
        self.aleph = 1.0        #必ず非満足状態になるように

    def initialize(self):
        self._V = np.array([0.5] * self._K)
        self._n = np.array([sys.float_info.min] * self._K)      #RS式でゼロ除算を防ぐため
        self._N = 0
        self.RS = np.zeros(self._K)     #RSの初期値はゼロ？？

    def update(self, arm, reward):
        self._alpha = 1 / (1 + self._n[arm])
        self._V[arm] = (1 - self._alpha) * self._V[arm] + self._alpha * reward
        self._n[arm] += 1
        self._N += 1
        self.RS[arm] = self._n[arm] / self._N * (self._V[arm] - self.aleph)     #RS評価値の更新(これは後で分ける？)

    def update_aleph(self):
        #2本腕を想定
        max_V = np.amax(self._V)
        mI = np.where(self._V == max_V)[0]
        nmI = np.where(self._V != max_V)[0]
        self.aleph = max_V * (1 - (self._V[nmI]/self._V[mI]) * (self._n[nmI]/self._n[mI])) / (1 - self._n[nmI]/self._n[mI])
        # print(f'aleph: {self.aleph}')

    def select_arm(self):
        maxRS = np.amax(self.RS)
        maxIndex = np.where(self.RS == maxRS)[0]
        return random.choice(maxIndex)

In [200]:
class Simulator(object):
    
    def __init__(self, trial, step, K):
        self._agent = Agent(K)
        self._env = None
        self._prob = None
        self._trial = trial
        self._step = step
        self._regret = np.zeros((trial, step))

    def run(self):
        for t in range(self._trial):
            print(f'{t}:')
            self._env = Environment(K)
            self._prob = self._env.prob
            self._agent.initialize()
            for s in range(self._step):
                arm = self._agent.select_arm()
                reward = self._env.play(arm)
                self._agent.update(arm, reward)
                # print(f'arm: {arm}, reward: {reward}')
                self.calc_regret(t, s, arm)
            # print(f'{self._agent.RS}')
            self._agent.update_aleph()
        self.print_regret()

    def calc_regret(self, t, s, arm):
        self._regret[t][s] += max(self._prob) - self._prob[arm]
        if s != 0: self._regret[t][s] += self._regret[t][s-1]

    def print_regret(self):
        plt.plot(np.arange(self._step), np.mean(self._regret, axis=0), label='RS', linestyle='dashed')
        plt.xlabel('steps')
        plt.ylabel('regret')
        plt.legend()
        plt.show()

0:
1:
2:
3:
4:
5:
6:
7:
8:
9:
10:
11:
12:
13:
14:
15:
16:
17:
18:
19:


In [202]:
trial = 20
step = 10000
K = 2
sim = Simulator(trial, step, K)
sim.run()

In [ ]:
a = np.array([1,4,7,2,5])
print(np.where(a == 2))
print(np.amax(a))

In [ ]:
a = np.arange(10)
random.choice(a)

In [ ]:
K = 20
arm_prob = np.array([np.random.rand() for _ in range(K)])
arm_prob

In [ ]:
sys.float_info.min + 1